# Preface
A significant proportion of this project was inspired by videos from 3Blue1Brown. Along with that, this will come heavily inspired from Neural Networks and Deep Learning, the free online book. It can be reached [neuralnetworksanddeeplearning.com](http://neuralnetworksanddeeplearning.com/). This version is outdated so many components were changed into how I conceptually understand them.

In [8]:
# Imports
import random
import numpy as np
from itertools import permutations

In [9]:
class Network(object):
    """
    Creates a Neural Network in specified layers with implemented Neural Network Methods
    """
    def __init__(self, sizes : list[int]):
        """
        Initializes the Network with random, normally distributed, biases and weights.

        Args:
            sizes (list): List of the number of neurons in each layer
        """
        self.sizes = sizes
        self.biases = [np.random.randn(layer, 1) for layer in sizes[1:]]
        self.weights = [np.random.randn(layer, prev) for prev, layer in zip(sizes[:-1], sizes[1:])]
    
    
    
    def feedForward(self, inputs : list[np.ndarray]):
        """
        Returns the output of the network given the input list

        Args:
            inputs (list): Input to the lyaer
        """
        for bias, weight in zip(self.biases, self.weights):
            inputs = np.tanh(np.dot(weight, inputs) + bias)
        return inputs
    
    
    
    def SGD(self, training_data : list[tuple[np.ndarray, np.ndarray]], epochs : int, mini_batch_size : int, learningRate : float, test_data : list[tuple[np.ndarray, int]] =None) -> None:
        """
        Trains the network using Stoachastic Gradient Descent. 

        Args:
            training_data (list of tuples): Training Data for the network
            epochs (int): number of iterations to adjust network
            mini_batch_size (int): size of each batch to pass through
            learningRate (float): Amount to shift layers in backpropogration
            test_data (list, optional): Passed through test data. Defaults to None.
        """
        
        for i in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[j:j + mini_batch_size] for j in range(0, len(training_data), mini_batch_size)]
            mini_batches = [batch for batch in mini_batches if len(batch) == mini_batch_size]
            
            for batch in mini_batches:
                self.update_mini_batch(batch, learningRate)
                
            if test_data:
                print(f'Epoch {i + 1}: {self.evaluate(test_data)} / {len(test_data)}') 
            else:
                print(f'Epoch {i + 1} complete!')
        
        
        
    def update_mini_batch(self, mini_batch : list[tuple[np.ndarray, np.ndarray]], learningRate : float):
        """
        Updates the weights and biases using SGD and Back Propogation

        Args:
            mini_batch (list[tuple[np.ndarray, np.ndarray]]): list containing a subset of the training data
            learningRate (float): Amount by which to shift existing layers after backpropogration
        """
        new_biases = [np.zeros(b.shape) for b in self.biases] #array of zeros w/ same dimension [( 1, 2), (3, 4)] -> [(0, 0), (0, 0)] 
        new_weights = [np.zeros(w.shape) for w in self.weights]
        
        for x, y in mini_batch:
            change_new_biases, change_new_weights = self.backprop(x, y)
            new_biases = [cur + change for cur, change in zip(new_biases, change_new_biases)]
            new_weights = [cur + change for cur, change in zip(new_weights, change_new_weights)]
        
        self.weights = [weight - (learningRate / len(mini_batch) * new_weight) for weight, new_weight in zip(self.weights, new_weights)]
        self.biases = [bias - (learningRate / len(mini_batch) * new_biases) for bias, new_biases in zip(self.biases, new_biases)] 
        
        
    def backprop(self, x, y) -> tuple:
        """
        Return a tuple representing the gradient for the cost function. 

        Args:
            x (np.ndarray): Input array
            y (np.ndarray): Expected output array
        """
        new_biases = [np.zeros(b.shape) for b in self.biases] #array of zeros w/ same dimension [( 1, 2), (3, 4)] -> [(0, 0), (0, 0)] 
        new_weights = [np.zeros(w.shape) for w in self.weights]
        
        #feed forward
        activation = x
        activations = [x] #To store all activations in each layer
        zs = [] #To store all z vectors
        
        # Iterate across layers
        for bias, weight in zip(self.biases, self.weights):
            z = np.dot(weight, activation).reshape(-1, 1) + bias # Have to reshape to correct dimensionality
            zs.append(z)
            activation = np.tanh(z)
            activations.append(activation)
        
        # backwards pass
        delta = self.cost_derivative(activations[-1].transpose()[0], y).reshape(-1, 1) * (1 / np.cosh(zs[-1])) ** 2 # Find the cost of our result (sech^2 from the derivative of tanh)
        
        new_biases[-1] = delta
        new_weights[-1] = np.dot(delta, activations[-2].transpose())
        
        for layer in range(-2, -len(self.sizes), -1): # Using negative indexing
            z = zs[layer]
            sp = np.tanh(z)
            delta = np.dot(self.weights[layer + 1].transpose(), delta) * sp
            new_biases[layer] = delta
            if len(self.sizes) == 3: #Strange bug I encountered
                new_weights[layer] = np.dot(delta, [activations[layer - 1]])
            else:
                new_weights[layer] = np.dot(delta, activations[layer - 1].T)
            break
            
        return (new_biases, new_weights)
    
    def evaluate(self, test_data : list[tuple[np.ndarray, int]]):
        """
        Return the number of correct outputs

        Args:
            test_data (list[tuple[np.ndarray, np.ndarray]]): test data in the form of ([input], output)
        """
        test_results = [(np.argmax(self.feedForward(x)), y) for (x,y) in test_data]
        return sum(int(x + 0.01 > y and x - 0.01 < y) for (x, y) in test_results)
        
    
    def cost_derivative(self, output_activations, y):
        """
        Return vector of partial derivatives of cost with respect to layer for the 

        Args:
            output_activations (_type_): what we got
            y (_type_): what the goal was
        """
        return output_activations - y

In [13]:
# Lets test the network!
myNet = Network([20, 10, 4, 2]) # have 2 neurons, 3 neurons, 2 neurons.

def createTestData(nums : tuple[int]):
    if 1 in nums:
        return [1, 0]
    return [0, 1]

def convertToInputs1(lst: list) -> list:
    retList = [0] * 20
    for i in range(len(lst)):
        retList[10*i + lst[i]] = 1
    return retList

assert(convertToInputs1([1, 9]) == [0, 1, 0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

myData = [(convertToInputs1(x), np.array(createTestData(x))) for x in [(np.array([int(10 * random.random()), int(10 * random.random())])) for _ in range(10000)]]
testData = [(convertToInputs1(x), 1 if 1 in x else 0) for x in [(np.array([int(10 * random.random()), int(10 * random.random())])) for _ in range(100)]]

myNet.SGD(myData, epochs=100, mini_batch_size=30, learningRate=2, test_data=testData)

Epoch 1: 48 / 100
Epoch 2: 0 / 100
Epoch 3: 0 / 100
Epoch 4: 0 / 100
Epoch 5: 86 / 100
Epoch 6: 0 / 100
Epoch 7: 0 / 100
Epoch 8: 82 / 100
Epoch 9: 82 / 100
Epoch 10: 82 / 100
Epoch 11: 82 / 100
Epoch 12: 82 / 100
Epoch 13: 82 / 100
Epoch 14: 0 / 100
Epoch 15: 0 / 100
Epoch 16: 0 / 100
Epoch 17: 0 / 100
Epoch 18: 0 / 100
Epoch 19: 0 / 100
Epoch 20: 0 / 100
Epoch 21: 0 / 100
Epoch 22: 0 / 100
Epoch 23: 0 / 100
Epoch 24: 0 / 100
Epoch 25: 0 / 100
Epoch 26: 0 / 100
Epoch 27: 0 / 100
Epoch 28: 0 / 100
Epoch 29: 0 / 100
Epoch 30: 0 / 100
Epoch 31: 0 / 100
Epoch 32: 0 / 100
Epoch 33: 0 / 100
Epoch 34: 0 / 100
Epoch 35: 0 / 100
Epoch 36: 0 / 100
Epoch 37: 0 / 100
Epoch 38: 0 / 100
Epoch 39: 0 / 100
Epoch 40: 0 / 100
Epoch 41: 0 / 100
Epoch 42: 78 / 100
Epoch 43: 0 / 100
Epoch 44: 78 / 100
Epoch 45: 0 / 100
Epoch 46: 78 / 100
Epoch 47: 0 / 100
Epoch 48: 0 / 100
Epoch 49: 78 / 100
Epoch 50: 78 / 100
Epoch 51: 78 / 100
Epoch 52: 78 / 100
Epoch 53: 78 / 100
Epoch 54: 78 / 100
Epoch 55: 78 / 100
E

In [36]:
# Lets test the network!
myNet = Network([16, 32, 16, 8, 4]) # have 4 neurons, 4 neurons, 2 neurons.

# Here is my rule: numbers must be in order. Output should be how many numbers are in order by the end. Simple O(n) 
def numsInPlace(nums : tuple[int]) -> int:
    return sum([1 if i + 1 == nums[i] else 0 for i in range(len(nums))])

assert(numsInPlace(tuple([1, 3, 2, 4])) == 2)

def convertToInputs2(lst: list) -> list:
    retList = [0] * 16
    for i in range(len(lst)):
        retList[4*i + lst[i] - 1] = 1
    return retList

assert(convertToInputs2([1, 2, 3, 4]) == [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1])

myData = [(np.array(convertToInputs2(x)), np.array([1 if numsInPlace(x) == i else 0 for i in range(4)])) for x in list(permutations([1, 2, 3, 4], 4))]
testData = [(np.array(convertToInputs2(x)), numsInPlace(convertToInputs2(x)) - 1) for x in list(permutations([1, 2, 3, 4], 4)) if random.random() < (1/3)]
# #self, training_data : list[tuple[np.ndarray, np.ndarray]], epochs : int, mini_batch_size : int, learningRate : float, test_data : list[tuple[np.ndarray, int]] =None
myNet.SGD(myData, epochs=100, mini_batch_size=10, learningRate=1, test_data=testData)

Epoch 1: 0 / 11
Epoch 2: 0 / 11
Epoch 3: 0 / 11
Epoch 4: 0 / 11
Epoch 5: 0 / 11
Epoch 6: 0 / 11
Epoch 7: 0 / 11
Epoch 8: 0 / 11
Epoch 9: 0 / 11
Epoch 10: 0 / 11
Epoch 11: 0 / 11
Epoch 12: 0 / 11
Epoch 13: 0 / 11
Epoch 14: 0 / 11
Epoch 15: 0 / 11
Epoch 16: 0 / 11
Epoch 17: 0 / 11
Epoch 18: 0 / 11
Epoch 19: 0 / 11
Epoch 20: 0 / 11
Epoch 21: 0 / 11
Epoch 22: 0 / 11
Epoch 23: 0 / 11
Epoch 24: 0 / 11
Epoch 25: 0 / 11
Epoch 26: 0 / 11
Epoch 27: 0 / 11
Epoch 28: 0 / 11
Epoch 29: 0 / 11
Epoch 30: 0 / 11
Epoch 31: 0 / 11
Epoch 32: 0 / 11
Epoch 33: 0 / 11
Epoch 34: 0 / 11
Epoch 35: 0 / 11
Epoch 36: 0 / 11
Epoch 37: 0 / 11
Epoch 38: 0 / 11
Epoch 39: 0 / 11
Epoch 40: 0 / 11
Epoch 41: 0 / 11
Epoch 42: 0 / 11
Epoch 43: 0 / 11
Epoch 44: 0 / 11
Epoch 45: 0 / 11
Epoch 46: 0 / 11
Epoch 47: 0 / 11
Epoch 48: 0 / 11
Epoch 49: 0 / 11
Epoch 50: 0 / 11
Epoch 51: 0 / 11
Epoch 52: 0 / 11
Epoch 53: 0 / 11
Epoch 54: 0 / 11
Epoch 55: 0 / 11
Epoch 56: 0 / 11
Epoch 57: 0 / 11
Epoch 58: 0 / 11
Epoch 59: 0 / 11
Epoch 

# Analysis
It appears that this Neural Net doesn't effectively predict these patterns. I'm still curious about it's capabilities: thus, I plan to use it in some of my future projects. Perhaps using different inputs that are outside of just numbers would do a better job. The model was originally designed to predict handwritten numbers as according to the book. I'm sure it can be reworked using more inputs and an adjusted learning rate to be more accurate in application.

I found that using more standardized inputs ([1, 0, 0, 0, 0] rather than [5]) allowed the network to perform better. I'm still curious about it's capabilities and interested to see how it can be applied. 